## Import

In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary

import wandb
import random


In [2]:
# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda
2


## Utils

In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Custom Dataset

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        #img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        #mask_path = os.path.join(directory_path, mask_path[2:])
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

## Data Loader

In [5]:
transform = A.Compose(
    [   
        A.Resize(128, 128),
        A.Normalize(),
        ToTensorV2()
    ]
)

# samsung label 변경한 걸로 수행
source_dataset = CustomDataset(csv_file="/mnt/nas27/Dataset/2023_design_project_2/samsung/train_source_CL.csv", transform=transform)
source_dataloader = DataLoader(source_dataset, batch_size=64, shuffle=True, num_workers=4)

unlabeled_dataset = CustomDataset(csv_file="/mnt/nas27/Dataset/2023_design_project_2/samsung/val_source_Pseudo_DM.csv", transform=transform)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=8, shuffle=True, num_workers=4)

valid_dataset = CustomDataset(csv_file="/mnt/nas27/Dataset/2023_design_project_2/samsung/val_source_CL.csv", transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=4)


In [6]:
# for images, masks in tqdm(source_dataloader):
#     print(masks[0].dtype)
#     print(masks.dtype)

In [7]:
# 클래스(레이블) 수
num_classes = 13

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

## Define Model

In [8]:
import os
import cv2
from collections import OrderedDict
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import yaml
from albumentations.augmentations import transforms
from albumentations.core.composition import Compose, OneOf
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from tqdm import tqdm
from init_weights import init_weights
from data_loader import Nuclie_dataset
from utils import BCEDiceLoss, AverageMeter, count_params, iou_score


class Unet_block(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.conv2 = nn.Conv2d(mid_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)        
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        return out


class Nested_UNet(nn.Module):
    def __init__(self, num_classes, input_channels=3, deep_supervision=False):
        super().__init__()

        num_filter = [32, 64, 128, 256, 512]
        self.deep_supervision = deep_supervision
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        # DownSampling
        self.conv0_0 = Unet_block(input_channels, num_filter[0], num_filter[0])
        self.conv1_0 = Unet_block(num_filter[0], num_filter[1], num_filter[1])
        self.conv2_0 = Unet_block(num_filter[1], num_filter[2], num_filter[2])
        self.conv3_0 = Unet_block(num_filter[2], num_filter[3], num_filter[3])
        self.conv4_0 = Unet_block(num_filter[3], num_filter[4], num_filter[4])

        # Upsampling & Dense skip
        # N to 1 skip
        self.conv0_1 = Unet_block(num_filter[0] + num_filter[1], num_filter[0], num_filter[0])
        self.conv1_1 = Unet_block(num_filter[1] + num_filter[2], num_filter[1], num_filter[1])
        self.conv2_1 = Unet_block(num_filter[2] + num_filter[3], num_filter[2], num_filter[2])
        self.conv3_1 = Unet_block(num_filter[3] + num_filter[4], num_filter[3], num_filter[3])
       
        # N to 2 skip
        self.conv0_2 = Unet_block(num_filter[0]*2 + num_filter[1], num_filter[0], num_filter[0])
        self.conv1_2 = Unet_block(num_filter[1]*2 + num_filter[2], num_filter[1], num_filter[1])
        self.conv2_2 = Unet_block(num_filter[2]*2 + num_filter[3], num_filter[2], num_filter[2])

        # N to 3 skip
        self.conv0_3 = Unet_block(num_filter[0]*3 + num_filter[1], num_filter[0], num_filter[0])
        self.conv1_3 = Unet_block(num_filter[1]*3 + num_filter[2], num_filter[1], num_filter[1])

        # N to 4 skip
        self.conv0_4 = Unet_block(num_filter[0]*4 + num_filter[1], num_filter[0], num_filter[0])

        if self.deep_supervision:
            self.output1 = nn.Conv2d(num_filter[0], num_classes, kernel_size=1)
            self.output2 = nn.Conv2d(num_filter[0], num_classes, kernel_size=1)
            self.output3 = nn.Conv2d(num_filter[0], num_classes, kernel_size=1)
            self.output4 = nn.Conv2d(num_filter[0], num_classes, kernel_size=1)

        else:
            self.output = nn.Conv2d(num_filter[0], num_classes, kernel_size=1)

        # initialise weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init_weights(m, init_type='kaiming')
            elif isinstance(m, nn.BatchNorm2d):
                init_weights(m, init_type='kaiming')

    def forward(self, x):                    # (Batch, 3, 256, 256)

        x0_0 = self.conv0_0(x)               
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], dim=1))
        
        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], dim=1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], dim=1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], dim=1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], dim=1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], dim=1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], dim=1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], dim=1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], dim=1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], dim=1))

        if self.deep_supervision:
            output1 = self.output1(x0_1)
            output2 = self.output2(x0_2)
            output3 = self.output3(x0_3)
            output4 = self.output4(x0_4)
            output = (output1 + output2 + output3 + output4) / 4
        else:
            output = self.output(x0_4)

        return output

In [9]:
# 모델 생성
model = Nested_UNet(num_classes=13, input_channels=3, deep_supervision=False).to(device)

# 입력 이미지의 크기를 지정합니다.
input_size = (3, 256, 256)  # 예시 입력 이미지 크기

# 모델의 구조를 출력합니다.
summary(model, input_size=input_size, batch_size=16, device=str(device).lower())  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [16, 32, 256, 256]             896
       BatchNorm2d-2         [16, 32, 256, 256]              64
              ReLU-3         [16, 32, 256, 256]               0
            Conv2d-4         [16, 32, 256, 256]           9,248
       BatchNorm2d-5         [16, 32, 256, 256]              64
              ReLU-6         [16, 32, 256, 256]               0
        Unet_block-7         [16, 32, 256, 256]               0
         MaxPool2d-8         [16, 32, 128, 128]               0
            Conv2d-9         [16, 64, 128, 128]          18,496
      BatchNorm2d-10         [16, 64, 128, 128]             128
             ReLU-11         [16, 64, 128, 128]               0
           Conv2d-12         [16, 64, 128, 128]          36,928
      BatchNorm2d-13         [16, 64, 128, 128]             128
             ReLU-14         [16, 64, 1

In [10]:
# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [11]:
dummy_input = torch.randn(1, 3, 256, 256)
dummy_input = dummy_input.float().to(device)
print(dummy_input.shape)
output = model(dummy_input)
print(output.shape)  # 결과 확인


torch.Size([1, 3, 256, 256])
torch.Size([1, 13, 256, 256])


In [12]:
save_directory = "./weight/"
best_val_mIoU = 0
project_name = "231108_pseudo_loop"


ALPHA = 0
ALPHA_t = 0.0001

T1 = 5
T2 = 10

In [13]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project= project_name,
   
     # track hyperparameters and run metadata
    config={
        "learning_rate": 0.001,
        "architecture": "UNet++",
        "dataset": "Samsung",
        "epochs": 100,
     }
)

for epoch in range(100):  # 5 에폭 동안 학습합니다.
          
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    
    # 학습
    model.train()
    epoch_loss = 0
    
    for sourcedata, pseudodata in tqdm(zip(source_dataloader, unlabeled_loader)):

        images, masks = sourcedata[0].float().to(device), sourcedata[1].long().to(device)
        optimizer.zero_grad()
        outputs = model(images)
        # print("outputs size", outputs.shape)

        if ALPHA > 0 :
            # pseudo label에 대해 학습.
            pseudoimages = pseudodata[0].float().to(device)
            poutputs = model(pseudoimages)
            plabels = torch.softmax(poutputs, dim=1)
            plabels = torch.argmax(plabels, dim=1)

            # print("poutputs size", poutputs.shape)
            # print("plabels size", plabels.shape)

            loss = criterion(outputs, masks.squeeze(1)) + ALPHA*criterion(poutputs, plabels.squeeze(1))
        
        else:
            loss = criterion(outputs, masks.squeeze(1))
        
        # loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # 학습 완료




        # train 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()
        
        
        





        for class_id in range(num_classes):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            train_class_ious.append(iou)
            
    train_class_ious = np.array(train_class_ious).reshape(-1, num_classes)
    train_class_ious = np.mean(train_class_ious, axis=0)
    
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class {class_id} IoU: {iou:.4f}')
     
        
    # mIoU 계산
    train_mIoU = np.mean(train_class_ious)

    # 특정 epoch이상일 때 alpha값 키우고, 모델이 pseudo image를 저장함.
    if(epoch > T1) and (epoch < T2):
        ALPHA = ALPHA_t*(epoch - T1)/(T2 - T1)
        

    elif epoch >= T2:
        ALPHA = ALPHA_t

    # validation
    val_loss = 0
    val_class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)

            # validation loss 계산
            val_loss += criterion(outputs, masks.squeeze(1)).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(num_classes):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                val_class_ious.append(iou)
                
    val_class_ious = np.array(val_class_ious).reshape(-1, num_classes)
    val_class_ious = np.mean(val_class_ious, axis=0)
    
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class {class_id} IoU: {iou:.4f}')       
   
    # mIoU 계산
    val_mIoU = np.mean(val_class_ious)
    
        
    if best_val_mIoU < val_mIoU:
        best_val_mIoU = val_mIoU
        torch.save(model.state_dict(), save_directory + project_name + '_E' +str(epoch+1)+ "_S" + str(best_val_mIoU) + '.pt')
        print("saved weights")



    # 에폭마다 결과 출력
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {epoch_loss/len(source_dataloader)}, Train mIoU Score: {train_mIoU:.4f}")
    print(f"Validation Loss: {val_loss/len(valid_dataloader)}, Validation mIoU Score: {val_mIoU:.4f}")
    print("___________________________________________________________________________________________\n")
    
        # log metrics to wandb
    wandb.log({"train_acc": train_mIoU, "train_loss": epoch_loss/len(source_dataloader)})
    wandb.log({"val_acc": val_mIoU, "val_loss": val_loss/len(valid_dataloader)})
    
    
# # [optional] finish the wandb run, necessary in notebooks
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dibidimi. Use `wandb login --relogin` to force relogin


69it [01:02,  1.11it/s]


Class 0 IoU: 0.1547
Class 1 IoU: 0.2297
Class 2 IoU: 0.3037
Class 3 IoU: 0.3336
Class 4 IoU: 0.3287
Class 5 IoU: 0.3524
Class 6 IoU: 0.3610
Class 7 IoU: 0.3411
Class 8 IoU: 0.3618
Class 9 IoU: 0.3613
Class 10 IoU: 0.3717
Class 11 IoU: 0.3735
Class 12 IoU: 0.3838


100%|██████████| 15/15 [00:13<00:00,  1.15it/s]


Class 0 IoU: 0.3973
Class 1 IoU: 0.3111
Class 2 IoU: 0.2865
Class 3 IoU: 0.2866
Class 4 IoU: 0.3916
Class 5 IoU: 0.2934
Class 6 IoU: 0.3889
Class 7 IoU: 0.3217
Class 8 IoU: 0.3223
Class 9 IoU: 0.2779
Class 10 IoU: 0.3240
Class 11 IoU: 0.3489
Class 12 IoU: 0.3256
saved weights

Epoch1
Train Loss: 0.8138635551583939, Train mIoU Score: 0.3275
Validation Loss: 0.6501286824544271, Validation mIoU Score: 0.3289
___________________________________________________________________________________________



...


69it [01:03,  1.09it/s]


Class 0 IoU: 0.8060
Class 1 IoU: 0.8310
Class 2 IoU: 0.8150
Class 3 IoU: 0.8431
Class 4 IoU: 0.8017
Class 5 IoU: 0.8120
Class 6 IoU: 0.8325
Class 7 IoU: 0.8324
Class 8 IoU: 0.8152
Class 9 IoU: 0.7993
Class 10 IoU: 0.8179
Class 11 IoU: 0.8139
Class 12 IoU: 0.8078


100%|██████████| 15/15 [00:13<00:00,  1.14it/s]

Class 0 IoU: 0.5089
Class 1 IoU: 0.4550
Class 2 IoU: 0.4281
Class 3 IoU: 0.4376
Class 4 IoU: 0.5126
Class 5 IoU: 0.4029
Class 6 IoU: 0.5100
Class 7 IoU: 0.4497
Class 8 IoU: 0.4496
Class 9 IoU: 0.4055
Class 10 IoU: 0.4541
Class 11 IoU: 0.4434
Class 12 IoU: 0.4446

Epoch28
Train Loss: 0.07541558211264403, Train mIoU Score: 0.8175
Validation Loss: 0.5066994686921438, Validation mIoU Score: 0.4540
___________________________________________________________________________________________




16it [00:16,  1.39it/s]

In [ ]:
print("outputs size", outputs.size)
print("masks size", masks.shape)
print(type(masks))
print("poutputs size", poutputs.size)
print("plabels size", plabels.shape)
print(type(plabels))

outputs size <built-in method size of Tensor object at 0x7fdc2716e230>
masks size torch.Size([32, 128, 128])
<class 'torch.Tensor'>
poutputs size <built-in method size of Tensor object at 0x7fdc2703bc80>
plabels size (32, 128, 128)
<class 'numpy.ndarray'>


In [ ]:
# test_dataset = CustomDataset(csv_file="/mnt/nas27/Dataset/Samsung_DM/test.csv", transform=transform, infer=True)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)